# Avaliação 2

### Questão 1
Utilizando a base Titanic disponível no kaggle (https://www.kaggle.com/c/titanic/data) elabore uma solução utilizando dois algoritmos de aprendizagem de máquina do seu conhecimento para classificar se o passageiro tem ou não chance de sobreviver. Os resultados dessa questão deverão ser descritos detalhadamente no relatório.
* Lembre de fazer todo o pré-processamento, explicando suas decisões;
* Avalie os resultados usando diferentes métricas.
* Teste 3 variações de parâmetros dos métodos;

#### Pré-processamento

É dado início ao pré-processamento, importando as bibliotecas necessárias e instanciando os databases forneceidos através do pandas.

É importante entender que:
* __gs__ equivale à instância do database que contém as saídas (coluna *Survived*) corretas para os testes
* __tr__ equivale à instância do database que contém os treinos
* __ts__ equivale à instância do database que contém os testes

Vale lembrar que o database __gs__ refere-se ao __teste__, e que o __teste__ não pertence ao treino. Portanto será interessante unir os dois databases futuramente.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn import cluster, neighbors, svm, metrics, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

gs = pd.read_csv("data/gender_submission.csv",sep=",")
tr = pd.read_csv("data/train.csv",sep=",")
ts = pd.read_csv("data/test.csv",sep=",")

#### Avaliando a quantidade de itens em cada database

Para entender a quantidade de itens com o qual se está trabalhando, é feita uma leitura através do comando __*len()*__.

In [2]:
print("Tamanho do database de treino:", len(tr))
print("Tamanho do database de testes:", len(ts))
print("Tamanho do database que contém as saídas corretas do teste:", len(gs))

Tamanho do database de treino: 891
Tamanho do database de testes: 418
Tamanho do database que contém as saídas corretas do teste: 418


#### Unindo todos os databases

Para passarem pelo pré-processamento, é interessante que todos os dados estejam juntos. Desta forma o __gs__ será unido ao __ts__, que será, por sua vez, unido ao __tr__.

❗️ __Utilizaremos o termo *dfTotal* para se referir ao DataFrame completo, que inclui o teste e o treino.__ ❗️

In [3]:
dfTotal = pd.concat([tr,pd.concat([gs, ts.drop(columns=['PassengerId'])], axis=1)])

#### Analisando quantos itens nulos há no database

É importante conhecer quantos itens nulos existem no database para entender qual deve ser a medida adequada a se tomar mediante a quantidade de itens nulos.

In [4]:
print("-->Nulidade:")
print(dfTotal.isnull().sum())

-->Nulidade:
PassengerId       0
Survived          0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64


⚠️ __Os itens nulos de cabine representam, no treino, mais de 77% do total de informações. Excluir todas essas informações pode representar uma alta perda ao treinamento do IA, ao passo que completar todo esse conteúdo com informações aleatórias ou fixas também podem trazer algum nível de imprecisão aos testes. Desta forma, optou-se por excluir essa coluna.__ ⚠️

#### Excluindo a coluna de Cabine

Como dito acima, a coluna *Cabin* apresenta muitos valores nulos. Desta forma passa a ser interessante excluir essa coluna.

In [5]:
dfTotal = dfTotal.drop(columns=['Cabin'])

#### Analisando quantos itens nulos há no database

Ainda analisando a quantidade de elementos nulos, será feita a análise novamente agora sem a coluna de cabine, que previamente apresentou várias informações faltantes.

In [6]:
print("-->Nulidade:")
print(dfTotal.isnull().sum())

-->Nulidade:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            263
SibSp            0
Parch            0
Ticket           0
Fare             1
Embarked         2
dtype: int64


❗️ __Considerando que *Fare* (Tarifa) e *Embarked* apresentam uma quantidade ínfima de valores nulos, considerou-se interessante, na coluna Fare, considerar a mediana no lugar do valor nulo e, no Embarked, o uso da moda dos valores.__ ❗️

#### Substituindo valores de Fare, Age e Embarked

Será considerada a moda dos valores de __Embarked__ para substituir os valores nulos e a mediana de __Fare__ e __Age__ para substituir os valores faltantes.

Note que, por mais que haja uma quantidade mais expressiva de valores nulos para *Age*, isso representa apenas 20%, da quantidade total. Sendo assim é entendeu-se como viável substituir os valores de idade pela mediana de todas as idades.

In [7]:
dfTotal["Fare"].fillna(dfTotal["Fare"].median(), inplace=True)
print(dfTotal.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            263
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64


In [8]:
dfTotal["Embarked"].fillna(dfTotal["Embarked"].mode()[0],inplace=True)
print(dfTotal.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            263
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         0
dtype: int64


In [9]:
dfTotal["Age"].fillna(dfTotal["Age"].median(),inplace=True)
print(dfTotal.isnull().sum())

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


#### Removendo colunas

Algumas colunas não auxiliam, pelo contrário, atrapalham o funcionamento do IA por apresentarem valores únicos para cada entidade. Desta forma, devem ser removidas as colunas:
* Name
* Ticket
* PassengerId


In [10]:
dfTotal = dfTotal.drop(columns=['Name','Ticket','PassengerId'])

#### Linhas duplicadas

Com a remoção das colunas, algumas linhas possuem valores exatamente iguais, não agregando em nada ao IA. Sendo assim, primeiramente será feita a análise de quantas linhas estão duplicadas:

In [11]:
print("Total de duplicações no database:",dfTotal.duplicated().sum())

Total de duplicações no database: 209


#### Remoção de linhas duplicadas

Existem 209 linhas duplicadas. Agora é interessante apagar essas duplicações que não agregam em nada ao IA.

In [12]:
dfTotal = dfTotal.drop_duplicates()
print("Conferindo o total de duplicações após apagar as linhas duplicadas:",dfTotal.duplicated().sum())

Conferindo o total de duplicações após apagar as linhas duplicadas: 0


#### Substituindo valores

Como um dos métodos a ser utilizado na análise será o KNN, então é necessário que todas as colunas sejam numéricas. Sendo assim, as colunas "Sex" e "Embarked" serão categorizadas em números:
* __Sex__
    * *male* passa a ser 0
    * *female* passa a ser 1
* __Embarked__
    * *C* passa a ser 0
    * *Q* passa a ser 1
    * *S* passa a ser 2

In [26]:
dfTotal['Sex'] = dfTotal['Sex'].replace('male',0)
dfTotal['Sex'] = dfTotal['Sex'].replace('female',1)

dfTotal['Embarked'] = dfTotal['Embarked'].replace('C',0)
dfTotal['Embarked'] = dfTotal['Embarked'].replace('Q',1)
dfTotal['Embarked'] = dfTotal['Embarked'].replace('S',2)

#### Analisando integridade dos dados

É importante saber se os dados estão dispostos de forma íntegra. Uma das análises realizadas no presente trabalho será analisar, caso a caso, se a disposição de categorias para cada coluna está feita da maneira correta. 
* Para a coluna __"Survived"__ os valores possíveis são 0 e 1
* Para a coluna __"Sex"__ os valores possíveis são 0 e 1
* Para a coluna __"Pclass"__ os valores possíveis são 1, 2 e 3
* Para a coluna __"Embarked"__ os valores possíveis são 0, 1 e 2

⚠️ __A coluna *Age* representa a idade de cada tripulante. Desta forma não faz sentido categorizar a idade, muito embora ela seja analisada de forma matemática mais adiante. Da mesma forma, a coluna *Fare* representa o preço e também será analisado de forma matemática mais adiante. Parch e SibSp representam, respectivamente número de pais/filhos tripulantes e número de irmãos/cônjuges a bordo, e, desta forma, também não é interessante tratá-los de forma categórica.__ ⚠️


In [31]:
print("Analisando valores de Survived:")
dfTotal.groupby(['Survived']).count()

Analisando valores de Survived:


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
Survived,,,,,,,
0,644,644,644,644,644,644,644
1,456,456,456,456,456,456,456


In [32]:
print("Analisando valores de Sex:")
dfTotal.groupby(['Sex']).count()

Analisando valores de Sex:


,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked
Sex,,,,,,,
0,672,672,672,672,672,672,672
1,428,428,428,428,428,428,428


In [33]:
print("Analisando valores de Pclass:")
dfTotal.groupby(['Pclass']).count()

Analisando valores de Pclass:


,Survived,Sex,Age,SibSp,Parch,Fare,Embarked
Pclass,,,,,,,
1,311,311,311,311,311,311,311
2,238,238,238,238,238,238,238
3,551,551,551,551,551,551,551


In [34]:
print("Analisando valores de Embarked:")
dfTotal.groupby(['Embarked']).count()

Analisando valores de Embarked:


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
Embarked,,,,,,,
0,247,247,247,247,247,247,247
1,84,84,84,84,84,84,84
2,769,769,769,769,769,769,769


✅ __As 4 colunas listadas estão categorizadas de forma adequada, não havendo nenhuma linha categorizada com uma classe que não exista.__ ✅